In [47]:
import pandas as pd
import numpy as np
import backtest
import matplotlib.pyplot as plt 

from log_data import StockData

import os

from dotenv import load_dotenv
from log_data import StockData
from datetime import datetime

import sys
import backtest

# Baixando dados das ações
df = pd.read_parquet('cotacoes_acoes.parquet')
df_ibov = pd.read_parquet('ibov.parquet')
df_ibov.rename(columns={'indice': 'ticker', 'valor': 'preco_fechamento_ajustado'}, inplace=True)
df_ibov['ticker'] = df_ibov['ticker'].replace('IBOV', 'BOVA11')
df_smal =  pd.read_parquet('smll.parquet')
df_smal.rename(columns={'indice': 'ticker', 'valor': 'preco_fechamento_ajustado'}, inplace=True)
df_smal['ticker'] = df_smal['ticker'].replace('SMLL', 'SMAL11')
df_ibov['data'] = pd.to_datetime(df_ibov['data']); df_smal['data'] = pd.to_datetime(df_smal['data'])
df = pd.concat([df, df_ibov, df_smal], ignore_index=True)
df.fillna(0, inplace=True)
df.info()

## upload data
load_dotenv()
ambiente_local = os.getenv('AMBIENTE_MAC')
datapath_cdi = 'cdi_historico.xlsx'
cdi_data = backtest.cdidata_extract(ambiente_local+datapath_cdi)
print(type(cdi_data))
print(len(cdi_data))
print(cdi_data[0:20])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650895 entries, 0 to 1650894
Data columns (total 12 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   data                       1650895 non-null  datetime64[ns]
 1   preco_abertura             1650895 non-null  float64       
 2   preco_fechamento           1650895 non-null  float64       
 3   preco_maximo               1650895 non-null  float64       
 4   preco_medio                1650895 non-null  float64       
 5   preco_minimo               1650895 non-null  float64       
 6   quantidade_negociada       1650895 non-null  float64       
 7   quantidade_negocios        1650895 non-null  float64       
 8   ticker                     1650895 non-null  object        
 9   volume_negociado           1650895 non-null  float64       
 10  fator_ajuste               1650895 non-null  float64       
 11  preco_fechamento_ajustado  1650895 no

In [48]:
# Dados ativos mais líquidos mes a mes
def valores_unicos_df(dataframe):
    valores_unicos = []
    for coluna in dataframe.columns[1:]:
        valores_unicos_coluna = list(dataframe[coluna].unique())
        valores_unicos.extend(valores_unicos_coluna)
    valores_unicos = list(set(valores_unicos))

    return valores_unicos

# Função para extrair o ano (primeiros 4 caracteres)
def extrair_ano(data):
    return data[:4]

# Função para extrair o valor após o hífen
def extrair_apos_hifen(data):
    partes = data.split('-')
    if len(partes) > 1:
        return partes[1]
    else:
        return None  

portfolio_liquidez = pd.read_csv('portfolio_quant.csv', delimiter=',', header=None)
# tickers_strategy = valores_unicos_df(portfolio_liquidez)
portfolio_liquidez.tail(5)
portfolio_liquidez.shape
portfolio_liquidez = portfolio_liquidez.iloc[:, 0:5]
portfolio_liquidez['Ano'] = portfolio_liquidez[0].apply(extrair_ano)
portfolio_liquidez['Mes'] = portfolio_liquidez[0].apply(extrair_apos_hifen)
tickers_strategy = valores_unicos_df(portfolio_liquidez)
portfolio_liquidez.head(5)


,0,1,2,3,4,Ano,Mes
0,2000-1,PETR4,BBDC4,ELET6,CMIG4,2000,1
1,2000-2,PETR4,CMIG4,BBDC4,ELET6,2000,2
2,2000-3,PETR4,CMIG4,BBDC4,ELET6,2000,3
3,2000-4,PETR4,BBDC4,CMIG4,ELET6,2000,4
4,2000-5,PETR4,BBDC4,CMIG4,ELET6,2000,5


In [49]:
# Objeto dados das ações
stock_data = StockData(df, tickers_selecionados=tickers_strategy)

stock_data.dates = stock_data.dates[:-2]
stock_data.price_matrix = stock_data.price_matrix[:-2, :]

print(len(tickers_strategy))
print(type(tickers_strategy))
print(stock_data.price_matrix.shape)
valores_nao_comuns = [valor for valor in tickers_strategy if valor not in stock_data.tickers]
weekdays = [StockData.date_to_weekday(date) for date in stock_data.dates]
print(weekdays[:5])
print(valores_nao_comuns)

# vamos colocar nome aos bois
# falar com o joao: acho melhor pegar as 20 mais do IBOVESPA. pq vem muito lixo nesse sistema que ele criou.
# > dates = dias 
# > tickers = ações
# > price matrix = precos
# > weekdays = dias da semana
# > tickers_strategy = todas as ações utilizadas na estratégia 
# > portfolio_liquidez = df com as ações mais líquidas de cada mês

# o que farei primeiro é tentar usar o dataframe para rodar o backtest
# verificar as saídas minuciosamente
# aumentar o escopo
# finalizar o programa - analisar volatilidade das quedas

# depois tentar rodar com numpy arrays
# depois colocar em classes e funções
# lembrar dos algorimos de jogos

61
<class 'list'>
(3424, 25)
['segunda', 'terça', 'quarta', 'quinta', 'sexta']
['2013', '5', '2007', '2011', '2004', '2014', '2001', '12', '8', '2017', '2010', '6', '2021', '3', '2008', '2002', '2016', '2020', '2022', '9', '2012', '2', '2003', '2018', '2006', '2019', '2015', '7', '10', '2005', '2009', '2023', '1', '4', '11', '2000']


In [50]:
# Verificação Ativos Início e Fim
assets_trading = 'Ativos backtest: '
for ticker in stock_data.tickers:
    assets_trading += ticker+' , '

print(assets_trading, '\n')
print('Inicio Backtest: ', stock_data.dates[0])
print('Fim do Backtest: ', stock_data.dates[-1])
print('Return Benchmark _ Strategy Long Bova11: ', round( stock_data.price_matrix[-1,stock_data.tickers.index('BOVA11')]/ stock_data.price_matrix[0,stock_data.tickers.index('BOVA11')] - 1, 2))

Ativos backtest: VALE3 , PETR3 , HYPE3 , PETR4 , LIGT3 , ELET6 , ELET3 , BRKM5 , TUPY3 , GGBR4 , USIM5 , BRAP4 , SANB11 , ITUB4 , CSNA3 , CPLE6 , BBAS3 , BBDC4 , CMIG4 , CIEL3 , MGLU3 , BBSE3 , ABEV3 , B3SA3 , BOVA11 ,  

Inicio Backtest:  2010-01-04 00:00:00
Fim do Backtest:  2023-10-27 00:00:00
Return Benchmark _ Strategy Long Bova11:  0.62


In [59]:
# backtest
contagem = np.zeros(stock_data.price_matrix.shape[1])
capital_alocacao = np.zeros((stock_data.price_matrix.shape[0], stock_data.price_matrix.shape[1] + 1))
resultado_alocacao = np.zeros((stock_data.price_matrix.shape[0], stock_data.price_matrix.shape[1] + 1))
capital_alocacao[:, -1] = 1

# Alocação portfolio
retorno_treshold = 0.01
periodo_position = 100 # input("Nr de dias posicionado: ")
cash = 1


for i, day in enumerate(stock_data.dates[:20]):

    # identificar mes e ano
    ano = day.year; mes = day.month
    
    # pegar a linha do joao com o portfolio desejado
    stocks2trade = portfolio_liquidez[(portfolio_liquidez['Ano'] == str(ano)) & (portfolio_liquidez['Mes'] == str(mes))]    
    if not stocks2trade.empty:
        stocks2trade = stocks2trade.iloc[0,1:-2].tolist()
    else:
        stocks2trade = []
        print('Falta mes e ano na portfolio de liquidez')
        sys.exit()        

    print(f'\nAno: {ano} e mês: {mes}')
    # print(stocks2trade)

    for i2, tickers in enumerate(stocks2trade):
            
        print(f'Ação {i2} : {tickers}')
        if weekdays[i] == 'sexta':

            preco_ontem = stock_data.price_matrix[i-1, stock_data.tickers.index(tickers)]
            print(weekdays[i-1])
            preco_sextapassada = stock_data.price_matrix[i-4, stock_data.tickers.index(tickers)]
            print(weekdays[i-4])
            # retorno_semanal = backtest.calculate_return()

            pass


Ano: 2010 e mês: 1
Ação 0 : PETR4
Ação 1 : TUPY3
Ação 2 : ITUB4
Ação 3 : GGBR4

Ano: 2010 e mês: 1
Ação 0 : PETR4
Ação 1 : TUPY3
Ação 2 : ITUB4
Ação 3 : GGBR4

Ano: 2010 e mês: 1
Ação 0 : PETR4
Ação 1 : TUPY3
Ação 2 : ITUB4
Ação 3 : GGBR4

Ano: 2010 e mês: 1
Ação 0 : PETR4
Ação 1 : TUPY3
Ação 2 : ITUB4
Ação 3 : GGBR4

Ano: 2010 e mês: 1
Ação 0 : PETR4
quinta
segunda
Ação 1 : TUPY3
quinta
segunda
Ação 2 : ITUB4
quinta
segunda
Ação 3 : GGBR4
quinta
segunda

Ano: 2010 e mês: 1
Ação 0 : PETR4
Ação 1 : TUPY3
Ação 2 : ITUB4
Ação 3 : GGBR4

Ano: 2010 e mês: 1
Ação 0 : PETR4
Ação 1 : TUPY3
Ação 2 : ITUB4
Ação 3 : GGBR4

Ano: 2010 e mês: 1
Ação 0 : PETR4
Ação 1 : TUPY3
Ação 2 : ITUB4
Ação 3 : GGBR4

Ano: 2010 e mês: 1
Ação 0 : PETR4
Ação 1 : TUPY3
Ação 2 : ITUB4
Ação 3 : GGBR4

Ano: 2010 e mês: 1
Ação 0 : PETR4
quinta
segunda
Ação 1 : TUPY3
quinta
segunda
Ação 2 : ITUB4
quinta
segunda
Ação 3 : GGBR4
quinta
segunda

Ano: 2010 e mês: 1
Ação 0 : PETR4
Ação 1 : TUPY3
Ação 2 : ITUB4
Ação 3 : GGBR4


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650895 entries, 0 to 1650894
Data columns (total 12 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   data                       1650895 non-null  datetime64[ns]
 1   preco_abertura             1650895 non-null  float64       
 2   preco_fechamento           1650895 non-null  float64       
 3   preco_maximo               1650895 non-null  float64       
 4   preco_medio                1650895 non-null  float64       
 5   preco_minimo               1650895 non-null  float64       
 6   quantidade_negociada       1650895 non-null  float64       
 7   quantidade_negocios        1650895 non-null  float64       
 8   ticker                     1650895 non-null  object        
 9   volume_negociado           1650895 non-null  float64       
 10  fator_ajuste               1650895 non-null  float64       
 11  preco_fechamento_ajustado  1650895 no

In [54]:
print(stock_data.tickers.index('PETR4'))
stock_data.price_matrix[-1, stock_data.tickers.index('PETR4')]

3


35.44

In [57]:
# plt.plot(stock_data.price_matrix[:,3])
print(stock_data.price_matrix.shape)
print(len(stock_data.dates))
print(len(stock_data.tickers))

(3424, 25)
3424
25
